# Projeto 06 - Safra - Parte 03 😎

**Integrantes:**
- Ricardo Meca
- Youlia Saito
- Karime Kerdahi
- Carolina Vieira
- Jaiana Santos
- Leonardo Tamayose


# Consumo de Energia

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as stats
import pandas as pd
import seaborn as sns
import statsmodels.api as sm
import statsmodels.tsa.api as smt

from dateutil.parser import parse
from datetime import datetime

from statsmodels.formula.api import ols
from statsmodels.stats.anova import anova_lm
from statsmodels.tsa.arima_model import ARIMA
from statsmodels.tsa.holtwinters import SimpleExpSmoothing, ExponentialSmoothing
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.statespace.sarimax import SARIMAX

sns.set_context('notebook')
sns.set_style('darkgrid')

In [ ]:
path_data = './ipeadata_consumo_energia.csv'
df = pd.read_csv(path_data, sep=';', skiprows=0)

data_dia = pd.Series(df['Data'])
consumo = pd.Series(df['Energia elétrica - consumo - residência - quantidade - GWh - Eletrobras - ELETRO12_CEERES12'])
df = pd.DataFrame({'Data': data_dia, 'Consumo': consumo})

In [ ]:
# Transformando os dados da coluna "Data" em string
df["Data"] = df["Data"].astype('str')

# Dividindo a data em dias e meses
df[['Ano', 'Mes']] = df['Data'].str.split('.', expand=True)

df['AnoMes'] = pd.to_datetime(df['Ano'] + df['Mes'], format='%Y%m')
df['AnoMes'] = df['AnoMes'].astype('object')
df.drop('Data', axis=1, inplace=True)
df["Consumo"] = df["Consumo"].astype(str)
df['Consumo'] = df['Consumo'].str.replace(',', '.').astype(float)
df['Ano'] = pd.to_numeric(df['Ano'])
df['Mes'] = pd.to_numeric(df['Mes'])

In [ ]:
df["Consumo"].mean()

In [ ]:
plt.figure(figsize=(15,10))
sns.scatterplot(x='AnoMes', y='Consumo', data=df, hue='Consumo', sizes=(20, 200))
plt.xlabel('Data', size = 14)
plt.ylabel('Consumo [GWh]', size = 14)
plt.title('Gráfico de dispersão do Consumo em função do tempo', size = 20)

In [ ]:
valor_min = df["Consumo"].describe()[3]
valor_max = df["Consumo"].describe()[7]
print(f"O valor mínimo e máximo são: {valor_min} GWh e {valor_max} GWh")

### Observações sobre o gráfico acima:
> - Atipicidade no gráfico: Há 20 anos, ocorreu um racionamento de energia na maior parte do país, com a redução compulsória de 20% do consumo de eletricidade.<br>
    Analisando o gráfico, por volta dos anos 2000, o eixo y tinha o valor aproximado de 7500 GWh, e com a redução do consumo de energia em aproximadamente 20%, há um gap no nosso gráfico, cujo valor cai para 6000 GWh aproximadamente.<br>
    
> - Os componentes da série temporal são: eixo Y - Consumo em GWh e o eixo X - Data. <br>

> - Os valores mínimo e máximo são, respectivamente: 1161.6 GWh e 13926.0 GWh.

In [ ]:
df['Consumo'].plot(kind='hist', bins=20, density=True, alpha=0.5, color='blue')
plt.title('Histograma da frequência do Consumo', size = 20)
plt.xlabel('Consumo [GWh]', size = 14)
plt.ylabel('Frequência', size = 14)

In [ ]:
sns.boxplot(x=df['Ano']//10 * 10, y='Consumo', data=df);

x=df['Ano']//10 * 10
plt.title('Boxplot', size = 20)
plt.xlabel('Década', size = 14)
plt.ylabel('Consumo [GWh]', size = 14)

## Sazonalidade, decomposição e resíduos

In [ ]:
def plot_df(df, x, y, title="", xlabel='Data', ylabel='Consumo [GWh]', dpi=100):
    plt.figure(figsize=(15,4), dpi=dpi)
    plt.plot(x, y, color='blue')
    plt.gca().set(title=title, xlabel=xlabel, ylabel=ylabel)
    plt.show()
    
x = df['AnoMes'].values
y1 = df['Consumo']
plot_df(df, x = x, y= y1, title='Sazonalidade')

In [ ]:
multiplicative_decomposition = seasonal_decompose(df['Consumo'], model='multiplicative', period=30)
additive_decomposition = seasonal_decompose(df['Consumo'], model='additive', period=30)
plt.rcParams.update({'figure.figsize': (16,12)})

multiplicative_decomposition.plot().suptitle('Decomposição Multiplicativa', fontsize=16)
plt.tight_layout(rect=[0, 0.03, 1, 0.95])

additive_decomposition.plot().suptitle('Decomposição Aditiva', fontsize=16)
plt.tight_layout(rect=[0, 0.03, 1, 0.95])
plt.show()

### Aqui, podemos observar que, ao analisarmos através do método multiplicativo, há um certo padrão nos resíduos mais definido do que no aditivo. No aditivo, os resíduos estão distribuidos de forma mais aleatória, o que faz essa decomposição ser a mais adequada. Podemos ainda observar que não há um comportamento normal nos resíduos.